In [1]:
# REQUIREMENTS
#   - see https://www.mosek.com/products/academic-licenses/ to get a .lic file and place it in the required directory
#   - then "pip install mosek"

import numpy as np
from state_space import generate_perturbed
from optimal_controls import *
import logging
logging.basicConfig(level=logging.INFO, force=True)

In [2]:
# n_states = 1
# n_inputs = 1
# A0 = np.array([[0.7]])
# B0 = np.array([[0.4]])
# n_plants = 2000
# A_list = generate_perturbed(A0, n_plants, 0.1)
# B_list = np.ones_like(A_list) * 0.4

In [3]:
# n_states = 2
# n_inputs = 2
# A0 = np.array([[1, 0.5],
#                [0.5, 1]])
# B0 = np.eye(2)
# n_plants = 128
# A_list = generate_perturbed(A0, n_plants, 0.1)
# B_list = generate_perturbed(B0, n_plants, 0.1)

In [12]:
n_states = 4
n_inputs = 1
A0 = np.array([[0, 0, 0, -0.048],
               [1, 0, 0, 0.676],
               [0, 1, 0, -2.21],
               [0, 0, 1, 2.6]])
B0 = np.array([[-0.12], [0.68], [-1.4], [1]])
n_plants = 128
A_list = generate_perturbed(A0, n_plants, 0.005)
B_list = generate_perturbed(B0, n_plants, 0.005)

In [13]:
# n_states = 6
# n_inputs = 1
# A0 = np.array([[0,  0,  0,  0,  0, -0.085],
#                [1,  0,  0,  0,  0,  0.642],
#                [0,  1,  0,  0,  0, -2.219],
#                [0,  0,  1,  0,  0,  4.354],
#                [0,  0,  0,  1,  0, -5.11 ],
#                [0,  0,  0,  0,  1,  3.4  ]])
# B0 = np.array([[-0.0068], [0.0796], [-0.4170], [1.17], [-1.7], [1]])
# n_plants = 256
# A_list = generate_perturbed(A0, n_plants, 0.005)
# B_list = generate_perturbed(B0, n_plants, 0.002)

In [14]:
K = get_optimal_controller(A_list, B_list)

INFO:optimal_controls: Defined 118 LMI constraints
INFO:optimal_controls: SolutionStatus.Optimal, simultaneously stabilising feedback gain is K = [[ 0.176 -0.387  0.325  1.827]]
INFO:optimal_controls: Optimised the simultaneously stabilising feedback gain, yielding K = [[ 0.097 -0.081  0.642  1.925]]
INFO:optimal_controls: Worst J using initial K: 72.1342
INFO:optimal_controls: Worst J using optimized K: 69.2228


In [15]:
ocSolver = OptimalControlSolver(A_list, B_list)
ocSolver.uncertainty_region_method = 'all_plants'
ocSolver.simstab_method = 'LMI'
ocSolver.minimize_wrt = 'all_plants'
ocSolver.solve()

INFO:optimal_controls: Defined 128 LMI constraints
INFO:optimal_controls: SolutionStatus.Optimal, simultaneously stabilising feedback gain is K = [[ 0.176 -0.387  0.325  1.827]]
INFO:optimal_controls: Optimised the simultaneously stabilising feedback gain, yielding K = [[ 0.097 -0.081  0.642  1.925]]
INFO:optimal_controls: Worst J using initial K: 72.1348
INFO:optimal_controls: Worst J using optimized K: 69.2228


In [16]:
ocSolver = OptimalControlSolver(A_list, B_list)
ocSolver.uncertainty_region_method = 'convhull_oliveira'
ocSolver.simstab_method = 'LMI'
ocSolver.minimize_wrt = 'all_plants'
ocSolver.solve()

INFO:optimal_controls: Convex hull of A set has 40 vertices
INFO:optimal_controls: Convex hull of B set has 50 vertices
INFO:optimal_controls: This will lead to 2000 LMI constraints
INFO:optimal_controls: Defined 2000 LMI constraints
INFO:optimal_controls: SolutionStatus.Optimal, simultaneously stabilising feedback gain is K = [[ 0.179 -0.386  0.326  1.828]]
INFO:optimal_controls: Optimised the simultaneously stabilising feedback gain, yielding K = [[ 0.097 -0.081  0.642  1.925]]
INFO:optimal_controls: Worst J using initial K: 72.1134
INFO:optimal_controls: Worst J using optimized K: 69.2228


In [17]:
ocSolver = OptimalControlSolver(A_list, B_list)
ocSolver.uncertainty_region_method = 'convhull_peaucelle'
ocSolver.simstab_method = 'LMI'
ocSolver.minimize_wrt = 'vertex_plants'
ocSolver.solve()

INFO:optimal_controls: Defined 118 LMI constraints
INFO:optimal_controls: SolutionStatus.Optimal, simultaneously stabilising feedback gain is K = [[ 0.176 -0.387  0.325  1.827]]
INFO:optimal_controls: Optimised the simultaneously stabilising feedback gain, yielding K = [[ 0.097 -0.081  0.642  1.925]]
INFO:optimal_controls: Worst J using initial K: 72.1342
INFO:optimal_controls: Worst J using optimized K: 69.2228


In [18]:
ocSolver = OptimalControlSolver(A_list, B_list)
ocSolver.uncertainty_region_method = 'all_plants'
ocSolver.simstab_method = 'riccati'
ocSolver.minimize_wrt = 'all_plants'
ocSolver.solve()

INFO:optimal_controls: Simultaneously stabilising feedback gain is K = [[ 0.097 -0.081  0.642  1.926]]
INFO:optimal_controls: Optimised the simultaneously stabilising feedback gain, yielding K = [[ 0.097 -0.081  0.642  1.926]]
INFO:optimal_controls: Worst J using initial K: 69.2228
INFO:optimal_controls: Worst J using optimized K: 69.2228


In [19]:
ocSolver = OptimalControlSolver(A_list, B_list)
ocSolver.uncertainty_region_method = 'lowresMVEE'
ocSolver.set_max_n_verts(1000)
ocSolver.simstab_method = 'LMI'
ocSolver.minimize_wrt = 'all_plants'
ocSolver.solve()

INFO:optimal_controls: Defined 510 LMI constraints
INFO:optimal_controls: SolutionStatus.Optimal, simultaneously stabilising feedback gain is K = [[0.561 0.055 0.867 2.310]]
INFO:optimal_controls: Optimised the simultaneously stabilising feedback gain, yielding K = [[ 0.097 -0.081  0.642  1.926]]
INFO:optimal_controls: Worst J using initial K: 80.3372
INFO:optimal_controls: Worst J using optimized K: 69.2228
